In [1]:
import pickle
from embedding import BertHuggingface
import math
from geometrical_bias import SAME, DirectBias, WEAT, RIPA, MAC, GeneralizedWEAT
import numpy as np
from lipstick import BiasGroupTest, NeighborTest, ClusterTest, ClassificationTest

/home/sarah/anaconda3/envs/nlp/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/sarah/anaconda3/envs/nlp/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


## Usage example

This is a minimialistic example on how to use the implemented bias scores. This includes reporting individual words' biases, 
biases for one set of neutral words (SAME, MAC, Direct Bias, RIPA) or several sets of neutral words representing different stereotypes (WEAT, generalized WEAT).

In [2]:
jobs = ['nurse', 'doctor', 'teacher', 'police officer', 'firefighter', 'secretary', 'programmer', 'engineer', 'caretaker', 'salesclerk']
jobs_m = ['doctor', 'police officer', 'firefighter', 'programmer', 'engineer', 'surgeon', 'rapper', 'businessman', 'pastor']
jobs_f = ['nurse', 'teacher', 'secetrary', 'caretaker', 'salesclerk', 'model', 'paralegal', 'dietitian', 'teacher']

jobs_black = ['taxi driver', 'basketball player']
jobs_white = ['police officer', 'lawyer']
jobs_asian = ['programmer', 'mathematician']

gender_attributes = [['he', 'man', 'his', 'boy', 'son', 'himself', 'father'], ['she', 'woman', 'her', 'girl', 'daughter', 'herself', 'mother']]
race_attributes = [['black', 'african'], ['white', 'caucasian'], ['asian', 'chinese']]

In [3]:
bert = BertHuggingface(2)

job_emb = bert.embed(jobs)
job_m_emb = bert.embed(jobs_m)
job_f_emb = bert.embed(jobs_f)
jobs_black_emb = bert.embed(jobs_black)
jobs_white_emb = bert.embed(jobs_white)
jobs_asian_emb = bert.embed(jobs_asian)
gender_attr = [bert.embed(attr) for attr in gender_attributes]
race_attr = [bert.embed(attr) for attr in race_attributes]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

WARNING! Using Bert on CPU!


## Defining the bias space

Each geometrical bias score implements the define_bias_space that takes an attribute set. The number of supported attribute groups depends on the score.
For the Direct Bias and RIPA one can set the number of bias dimensions k and parameter c that determines the strictness of bias measurements (see the paper/ implementation).

In [4]:
gweat = GeneralizedWEAT()
gweat.define_bias_space(gender_attr)

gweat2 = GeneralizedWEAT()
gweat2.define_bias_space(race_attr)

In [5]:
mac = MAC()
mac.define_bias_space(gender_attr)

In [6]:
weat = WEAT()
weat.define_bias_space(gender_attr)

In [7]:
same = SAME()
same.define_bias_space(gender_attr)

In [8]:
db1 = DirectBias(k=1,c=1)
db1.define_bias_space(gender_attr)

LinAlgError: SVD did not converge

In [9]:
db7 = DirectBias(k=7,c=1)
db7.define_bias_space(gender_attr)

LinAlgError: SVD did not converge

In [10]:
ripa1 = RIPA(k=1,c=1)
ripa1.define_bias_space(gender_attr)

LinAlgError: SVD did not converge

In [11]:
ripa7 = RIPA(k=7,c=1)
ripa7.define_bias_space(gender_attr)

LinAlgError: SVD did not converge

## Individual word biases

In [14]:
score_names = ['mac', 'db1', 'db7', 'ripa1', 'ripa7', 'same', 'weat']
scores = [mac, db1, db7, ripa1, ripa7, same, weat]

for i in range(len(scores)):
    print(score_names[i], ": ", [scores[i].individual_bias(emb) for emb in job_emb])


mac :  [0.1564862940992628, 0.13405894807406835, 0.20347282716206141, 0.3124208578041622, 0.42738594327654156, 0.2571887842246464, 0.22850877046585083, 0.3312168163912637, 0.23186555079051427, 0.4556000019822802]


TypeError: 'NoneType' object is not iterable

## Group biases

Most bias scores implement a mean bias over one set of words, WEAT requires several groups of words, matching the number of attribute groups. For WEAT only 2 groups are supported, the generalized WEAT can handle an arbitrary number.

In [15]:
# most scores implement a mean bias
for i in range(len(scores)-1):
    print(score_names[i], ": ", scores[i].mean_individual_bias(job_emb))
    
# weat implements an effect size over two groups stereotypically associated with the gender attribute groups
print("weat: ", weat.group_bias([job_m_emb, job_f_emb]))
print("gweat (gender): ", gweat.group_bias([job_m_emb, job_f_emb]))
print("gweat (race): ", gweat2.group_bias([jobs_black_emb, jobs_white_emb, jobs_asian_emb]))

mac :  0.27382047942706517


TypeError: 'NoneType' object is not iterable

## SAME

SAME implements additional functions to measure skew and stereotype and for multiclass bias (n>2) it can return the pairwise signed biases used to obtain the overall bias.

Skew and Stereotype are only implemented pairwise, so the user has to specify which attributes to use (according to the order of attribute groups).

In [16]:
same.define_bias_space(race_attr)
print("Black vs. White")
print("Skew: ", same.skew_pairwise(job_emb, 0, 1))
print("Stereotype: ", same.stereotype_pairwise(job_emb, 0, 1))
print()

print("Asian vs. White")
print("Skew: ", same.skew_pairwise(job_emb, 2, 1))
print("Stereotype: ", same.stereotype_pairwise(job_emb, 2, 1))
print()

same.define_bias_space(race_attr)
print("Multiclass bias vector for 'nurse': ", same.individual_bias_per_pair(job_emb[0])) # first is black/white, second black/asian
print("bias magntiude for 'nurse': ", same.individual_bias(job_emb[0]))

Black vs. White
Skew:  -0.0169088353980276
Stereotype:  0.08388088118690003

Asian vs. White
Skew:  -0.008156276582493164
Stereotype:  0.03200832200098757

Multiclass bias vector for 'nurse':  [0.11137423863006458, 0.07711813268048658]
bias magntiude for 'nurse':  0.1422651805362037


## Cluster, neighbor and classification test

In [17]:
neighborTest = NeighborTest(k=5)

# this is how the neighbor test is used in the paper:
# TODO: call weat on jobs, sort by bias into m/f groups
weats = [weat.individual_bias(emb) for emb in job_emb]
sort_idx = np.argsort(weats)
jobs_f_weat = [job_emb[idx] for idx in sort_idx[:5]]
jobs_m_weat = [job_emb[idx] for idx in sort_idx[-5:]]
print("bias by neighbor (as in the paper):")
print(neighborTest.bias_by_neighbor([jobs_f_weat, jobs_m_weat]))

# instead of using weat we can define stereotypical groups by hand
jobs_gender = [job_m_emb, job_f_emb]
print("bias by neighbor (without weat):")
print(neighborTest.bias_by_neighbor(jobs_gender))

# define the bias space with a subset of known stereotypical words, then test words without known categories
neighborTest.define_bias_space(gender_attr)
print("bias by neighbor (without predefined groups): ")
biases = [neighborTest.individual_bias(emb) for emb in job_emb]
print(biases)

print("mean bias by neighbor (without predefined groups): ")
print(neighborTest.mean_individual_bias(job_emb))

bias by neighbor (as in the paper):
[0.8, 1.0, 0.8, 1.0, 1.0, 0.2, 0.2, 0.2, 0.2, 0.4]
bias by neighbor (without weat):
[0.8, 0.4, 1.0, 1.0, 0.8, 0.8, 1.0, 0.8, 0.8, 0.2, 0.6, 0.6, 0.2, 0.6, 0.6, 0.6, 0.2, 0.6]
bias by neighbor (without predefined groups): 
[0.0, 1.0]
[0.2, 0.8]
[0.2, 0.8]
[0.4, 0.6]
[0.2, 0.8]
[0.4, 0.6]
[0.4, 0.6]
[0.4, 0.6]
[0.2, 0.8]
[0.4, 0.6]
[1.0, 0.6000000000000001, 0.6000000000000001, 0.19999999999999996, 0.6000000000000001, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.6000000000000001, 0.19999999999999996]
mean bias by neighbor (without predefined groups): 
[0.0, 1.0]
[0.2, 0.8]
[0.2, 0.8]
[0.4, 0.6]
[0.2, 0.8]
[0.4, 0.6]
[0.4, 0.6]
[0.4, 0.6]
[0.2, 0.8]
[0.4, 0.6]
0.4400000000000001


In [18]:
clusterTest = ClusterTest()

# according to the paper
weats = [weat.individual_bias(emb) for emb in job_emb]
sort_idx = np.argsort(weats)
jobs_f_weat = [job_emb[idx] for idx in sort_idx[:5]]
jobs_m_weat = [job_emb[idx] for idx in sort_idx[-5:]]
print("cluster test accuracy (weat): ")
print(clusterTest.cluster_test([jobs_f_weat, jobs_m_weat]))

# instead of using weat we can define stereotypical groups by hand
jobs_gender = [job_m_emb, job_f_emb]
print("cluster test accuracy (predefined groups): ")
print(clusterTest.cluster_test(jobs_gender))

# define the bias space with a subset of known stereotypical words, then test words without known categories
#clusterTest.define_bias_space(gender_attr)
#clusterTest.mean_individual_bias(job_emb)


cluster test accuracy (weat): 
0.6
cluster test accuracy (predefined groups): 
0.5185185185185186


In [20]:
clfTest = ClassificationTest()
cv_scores = clfTest.classification_test(jobs_gender)
print(np.mean(cv_scores), np.std(cv_scores))

0.6666666666666667 0.07302967433402217
